# Japan Search Gallery client

> Tools to interact with the Japan Search Gallery

In [1000]:
#| default_exp api

In [1001]:
#| hide
from nbdev.showdoc import *

In [1002]:
#| export
from pprint import pprint
import os
import json
import base64
import time

In [1003]:
#| export

class Component:

    item = {
        "parts": []
    }
    
    def export(self):
        '''
        エクスポート

        Returns:  
        * dict containing representation of the gallery
        '''
        return self.item

    def setLangMetadata(self, key, lang, value):
        '''
        メタデータの設定

        Parameters:  
        * `key` - title, description, etc...  
        * `lang` - one of 言語, eg: 'ja', 'en', 'yomi'  
        * `value` - 値
        '''
        item = self.item
        if key not in item:
            item[key] = {}
        item[key][lang] = value

    def setMetadata(self, key, data):
        self.item[key] = data

    def addPart(self, part):
        '''
        パーツの追加
        
        Parameters:
        * `part` - パーツ        
        '''
        item = self.item
        if 'parts' not in item:
            item['parts'] = []
        item["parts"].append(part.export())  

class Media(Component):

    def __init__(self, type, title, source):
        self.item = {
            "imgType": type,
            "cover": None,
            "sourceUrl": None,
            "title": {
                "ja": title
            },
            "source": {
                "ja": source
            },
            "contentsRightsType": "various",
        }

class JpsCurationIIIFImage(Media):
    def __init__(self, manifestUrl, infoJsonUrl, title, source, xywh=None):
        super().__init__("IIIF", title, source)

        item = self.item
        item["cover"] = None
        item["sourceUrl"] = None
        item["manifestUrl"] = manifestUrl
        item["infoJsonUrl"] = infoJsonUrl

        if xywh is not None:
            spl = xywh.split(',')
            item["x"] = int(spl[0])
            item["y"] = int(spl[1])
            item["w"] = int(spl[2])
            item["h"] = int(spl[3])

class JpsCurationUrlImage(Media):
    """
    URLを指定して表示する画像に関するクラス
    """

    def __init__(self, url, title, source, thumbnailUrl=None):
        """
        初期化

        Parameters:  
        * `url` - 画像URL  
        * `title` - 日本語のタイトル  
        * `source` - 日本語の出典  
        * `thumbnailUrl` - サムネイルURL
        """
        super().__init__("URL", title, source)

        item = self.item

        item["url"] = url
        if thumbnailUrl is not None:
            item["thumbnailUrl"] = thumbnailUrl



class Part(Component):

    def __init__(self, type, viewType=None): # , viewType
        item = {}
        self.item = item
        # item["cpid"] = "a"
        item["type"] = type

        if viewType is not None:
            item["viewType"] = viewType

    def addText(self, lang, value):
        '''
        opsに要素を追加します。

        Parameters:  
        * `lang` - one of 言語, eg: 'ja', 'en', 'yomi'  
        * `value` - 値
        '''
        item = self.item

        ops = item[lang]["ops"]
        ops.append({
            "insert": value
        })
        ops.append({
            "insert": "\n"
        })

    def setUrl(self, url):
        '''
        外部リンクのURLを設定します。

        Parameters:  
        * `url` - 外部リンクのURL
        '''
        self.item["url"] = url 

    def setImage(self, image):
        self.item["image"] = image.export()

class JpsCurationListExternalLink(Part):
    '''
    ListExternalLink
    '''
    
    def __init__(self):
        '''
        初期化
        '''
        super().__init__("jps-curation-list-external-link")

class JpsCurationTimeline(Part):
    '''
    Timeline
    '''
    
    def __init__(self, start, end, layout="bar"):
        '''
        初期化
        '''
        super().__init__("jps-curation-timeline")

        item = self.item
        item["timeline"] = {
            "start": start,
            "end": end,
        }
        item["layout"] = layout

class JpsCurationTimepoint(Part):
    '''
    Timepoint
    '''
    
    def __init__(self, name_ja, name_en, start, end, startFraction=0, endFraction=0, labelHeight=1):
        '''
        初期化
        '''
        super().__init__("jps-curation-timepoint")

        item = self.item
        item["time"] = {
            "name": {
                "ja": name_ja,
                "en": name_en,
            },
            "start": start,
            "end": end,
            "startFraction": startFraction,
            "endFraction": endFraction,
            "labelHeight": labelHeight
        }

class JpsCurationText(Part):
    '''
    Text
    '''
    
    def __init__(self):
        '''
        初期化
        '''
        super().__init__("jps-curation-text")

        item = self.item
        item["ja"] = {
            "ops": []
        }

        item["en"] = {
            "ops": []
        }

class JpsCurationList(Part):
    '''
    リストパーツ
    '''
    def __init__(self, viewType):
        '''
        初期化

        Parameters:  
        * `viewType` - レイアウト, eg: 'tile', 'table', etc...
        '''
        # super().__init__("list") # , viewType
        super().__init__("jps-curation-list") # , viewType

        item = self.item
        item["viewType"] = viewType

class JpsCuration(Part):

    def __init__(self, curationType="GALLERY", version=1, rightsType="ccby"):
        date = int(time.time() * 1000) 
        item = {
            "version": version,
            "title": {},
            "summary": {},
            "responsibility": {},
            "workspaceTags": [],
            "permission": {},
            "edit": {
                # "userId": "system",
                # "orgId": "jpsws",
                "date": date,
                "action": "CREATE"
            },
            "parts": [],
            "rights": {},
            "rightsType": rightsType,
            "curationType": curationType,
        }
        self.item = item

    def setTheme(self, theme):
        self.item["theme"] = theme

    def addTag(self, tag):
        '''
        タグを追加します。

        Parameters:  
        * `tag` - タグ
        '''
        item = self.item
        item["workspaceTags"].append(tag)

    def saveJsonl(self, path):
        '''
        JSONLファイルの保存

        Parameters:  
        * `path` - 出力先のパス
        '''

        arr = []
        arr.append(json.dumps(self.item, ensure_ascii=False))

        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, mode='w') as f:
            f.write("\n".join(arr))

    def save(self, path):
        '''
        JSONファイルの保存

        Parameters:  
        * `path` - 出力先のパス
        '''
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            json.dump(self.item, f, ensure_ascii=False, indent=2)

    def encode(self, curationType="NOTE"):
        '''
        Base64エンコード

        Returns:  
        * str containing Base64 encoded representation of the gallery
        '''
        item = self.item
        item["curationType"] = curationType
        json_string = json.dumps(item)
        return base64.b64encode(json_string.encode('utf-8')).decode('utf-8')

First import the JPSGalleryClient class.
```
from jps_gallery_tools.api import JPSGalleryClient
```
To initialise the JPSGalleryClient you need to provide the base url of your Omeka instance's API.

In [1004]:
gallery = JpsCuration()

## ギャラリーのメタデータの設定

ギャラリーのメタデータを設定します。

In [1005]:
show_doc(JpsCuration.setLangMetadata)

---

### Component.setLangMetadata

>      Component.setLangMetadata (key, lang, value)

メタデータの設定

Parameters:  
* `key` - title, description, etc...  
* `lang` - one of 言語, eg: 'ja', 'en', 'yomi'  
* `value` - 値

タイトルの設定

In [1006]:
gallery.setLangMetadata("title", "ja", "日本語タイトル")
gallery.setLangMetadata("title", "en", "English Title")
gallery.setLangMetadata("title", "yomi", "にほんごのよみ")

summaryの設定

In [1007]:
gallery.setLangMetadata("summary", "ja", "日本語の概要文")
gallery.setLangMetadata("summary", "en", "English Summary")

responsibilityの設定

In [1008]:
gallery.setLangMetadata("responsibility", "ja", "中村覚")
gallery.setLangMetadata("responsibility", "en", "Satoru Nakamura")

tagの設定

In [1009]:
show_doc(JpsCuration.addTag)

---

### JpsCuration.addTag

>      JpsCuration.addTag (tag)

タグを追加します。

Parameters:  
* `tag` - タグ

In [1010]:
gallery.addTag("はじめてのタグ")
gallery.addTag("二つ目のタグ")

画像の設定

In [1011]:
show_doc(JpsCurationUrlImage)

---

### JpsCurationUrlImage

>      JpsCurationUrlImage (url, title, source, thumbnailUrl=None)

URLを指定して表示する画像に関するクラス

In [1012]:
original_url = "https://blogger.googleusercontent.com/img/a/AVvXsEiTjjaWkCHeCslxeKujaQotjuR_HvhOHen7Ql7rQV8nmQl4ivLRF_CSabMY-JLJHNL-xHMe3VDyA9F-iv1RtQ2jFTk1Vnjck-veKLUZPdYqCcjzHkc8SGKb1fyZ_x8W66aTrSXQ78UN3CnyBakwwmmH_hmPKeEsTzq8J3e2zzUMylmDjCZ0LClPByBRRw=s400"
thumbnail_url = "https://blogger.googleusercontent.com/img/a/AVvXsEiTjjaWkCHeCslxeKujaQotjuR_HvhOHen7Ql7rQV8nmQl4ivLRF_CSabMY-JLJHNL-xHMe3VDyA9F-iv1RtQ2jFTk1Vnjck-veKLUZPdYqCcjzHkc8SGKb1fyZ_x8W66aTrSXQ78UN3CnyBakwwmmH_hmPKeEsTzq8J3e2zzUMylmDjCZ0LClPByBRRw=s180-c"
title = "初日の出のキャラクター"
source = "いらすとや"
top_image = JpsCurationUrlImage(original_url, title, source, thumbnail_url)

In [1013]:
show_doc(JpsCuration.setImage)

---

[source](https://github.com/nakamura196/jps_gallery_tools/blob/main/jps_gallery_tools/api.py#L187){target="_blank" style="float:right; font-size:smaller"}

### Part.setImage

>      Part.setImage (image)

In [1014]:
gallery.setImage(top_image)

## パーツの追加

In [1015]:
show_doc(JpsCurationList)

---

### JpsCurationList

>      JpsCurationList (viewType)

リストパーツ

In [1016]:
part = JpsCurationList("tile")

メタデータの追加

In [1017]:
part.setLangMetadata("title", "ja", "リストのタイトル")
part.setLangMetadata("title", "en", "List Title")

リストに画像の追加

In [1018]:
item = JpsCurationListExternalLink()

In [1019]:
show_doc(JpsCurationListExternalLink.setUrl)

---

[source](https://github.com/nakamura196/jps_gallery_tools/blob/main/jps_gallery_tools/api.py#L184){target="_blank" style="float:right; font-size:smaller"}

### Part.setUrl

>      Part.setUrl (url)

外部リンクのURLを設定します。

Parameters:  
* `url` - 外部リンクのURL

In [1020]:
item.setUrl("https://www.irasutoya.com/2021/11/blog-post_91.html")

アイテムの追加

In [1021]:
show_doc(JpsCurationList.addPart)

---

[source](https://github.com/nakamura196/jps_gallery_tools/blob/main/jps_gallery_tools/api.py#L53){target="_blank" style="float:right; font-size:smaller"}

### Component.addPart

>      Component.addPart (part)

パーツの追加

Parameters:
* `part` - パーツ

In [1022]:
part.addPart(item)

## Save json

設定した内容をjsonファイルに出力します。

In [1023]:
show_doc(JpsCuration.save)

---

### JpsCuration.save

>      JpsCuration.save (path)

JSONファイルの保存

Parameters:  
* `path` - 出力先のパス

In [1024]:
gallery.save("tmp/gallery.json")

In [1025]:
show_doc(JpsCuration.saveJsonl)

---

### JpsCuration.saveJsonl

>      JpsCuration.saveJsonl (path)

JSONLファイルの保存

Parameters:  
* `path` - 出力先のパス

In [1026]:
gallery.saveJsonl("tmp/gallery.jsonl")

In [1027]:
show_doc(JpsCuration.encode)

---

### JpsCuration.encode

>      JpsCuration.encode (curationType='NOTE')

Base64エンコード

Returns:  
* str containing Base64 encoded representation of the gallery

In [1028]:
#| hide
import nbdev; nbdev.nbdev_export()